In [1]:
!export LC_ALL=C.UTF-8 # в процессе работы в среде Colab возникают неожиданные проблемы с кодировкой, поэтому устанавливается принудительная необходимая кодировка символов
!export LANG=C.UTF-8
%pip install -U pip setuptools wheel # автоматическая настройка среды и зависимостей для SpaCy
%pip install -U 'spacy[cuda12x]' # Colab использует cuda 12.0
%pip install natasha
%pip install cairosvg # svg -> png, работает только на малых текстах
!python -m spacy download ru_core_news_sm

2023-12-18 23:47:46.507406: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 23:47:46.507460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 23:47:46.508965: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-18 23:47:46.516538: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-18 23:47:47.580050: W tensorflow/compiler/tf2

In [2]:
import spacy
import cairosvg
import os
from natasha import Segmenter, NewsEmbedding, NewsSyntaxParser, Doc
from spacy.lang.ru import Russian
from spacy import displacy
from pathlib import Path

spacy.prefer_gpu() #для ускорения работы с большими текстами в SpaCy, решил использовать GPU. На файле book1857_ok.txt скорость обработки увеличилась в 3 раза(В среде CoLab)
# ВАЖНО - При отсутствии GPU код не будет работать, т.к. принудительно включен режим работы с граф. ускорителем
nlp = spacy.load("ru_core_news_sm")
nlp.max_length = 1500000 #в одной из книг находится больше 1 млн символов, и обработчик упирается в лимит Spacy - поэтому пришлось увеличить ограничение вручную.

In [3]:
#файлы
file_paths = [
    "book1857_ok.txt",
    "book1863_ok.txt",
    "book1873_ok.txt",
    "book1876_ok.txt",
    "book1882_ok.txt",
    "book1887_ok.txt"
]

In [4]:
#обработка данных через Spacy
for file_path in file_paths:
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()

    doc_spacy = nlp(text)

    output_path_spacy_txt = f"analysis_spacy_{file_path[:-4]}.txt"
    with open(output_path_spacy_txt, "w", encoding="utf-8") as output_file_spacy:
        output_file_spacy.write(f"Синтаксический анализ (spaCy) для файла {file_path}:\n")
        for token in doc_spacy:
            output_file_spacy.write(f"{token.text} {token.pos_} {token.dep_}\n")

    svg = displacy.render(doc_spacy, style="dep", jupyter=False)
    file_name = f"Visual_analysis_spacy_{file_path[:-4]}.svg"
    output_path = Path("SVG" + file_name)
    output_path.open("w", encoding="utf-8").write(svg)


    print(file_path + " проанализирован через spaCy. Анализ сохранен в новый TXT файл и также в SVG и PNG формате визуализации")

book1857_ok.txt проанализирован через spaCy. Анализ сохранен в новый TXT файл и также в SVG и PNG формате визуализации
book1863_ok.txt проанализирован через spaCy. Анализ сохранен в новый TXT файл и также в SVG и PNG формате визуализации
book1873_ok.txt проанализирован через spaCy. Анализ сохранен в новый TXT файл и также в SVG и PNG формате визуализации
book1876_ok.txt проанализирован через spaCy. Анализ сохранен в новый TXT файл и также в SVG и PNG формате визуализации
book1882_ok.txt проанализирован через spaCy. Анализ сохранен в новый TXT файл и также в SVG и PNG формате визуализации
book1887_ok.txt проанализирован через spaCy. Анализ сохранен в новый TXT файл и также в SVG и PNG формате визуализации


In [5]:
#попытка сохранить SVG в PNG файл, но к сожалению файлы слишком большие
#будет работать на более коротких принимаемых текстах в SpaCy
current_directory = os.getcwd()

for svg_filename in os.listdir(current_directory):
    if svg_filename.endswith(".svg"):

        svg_path = os.path.join(current_directory, svg_filename)
        png_filename = svg_filename[:-4] + ".png"
        png_path = os.path.join(current_directory, png_filename)

        cairosvg.svg2png(url=svg_path, write_to=png_path)


CairoError: ignored

In [6]:
# обработка данных через natasha
segmenter = Segmenter()
emb = NewsEmbedding()
syntax_parser = NewsSyntaxParser(emb)

for file_path in file_paths:
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()

    doc_natasha = Doc(text)
    doc_natasha.segment(segmenter)
    doc_natasha.parse_syntax(syntax_parser)

    output_path_natasha_txt = f"analysis_natasha_{file_path[:-4]}.txt"

    with open(output_path_natasha_txt, "w", encoding="utf-8") as output_file_natasha:
        output_file_natasha.write(f"Синтаксический анализ (Natasha) для файла {file_path}:\n")
        for token in doc_natasha.tokens:
            output_file_natasha.write(f"DocToken(start={token.start}, stop={token.stop}, text='{token.text}')\n")

    print(file_path + " проанализирован через natasha и результат сохранен в новом TXT файле")

book1857_ok.txt проанализирован через natasha и результат сохранен в новом TXT файле
book1863_ok.txt проанализирован через natasha и результат сохранен в новом TXT файле
book1873_ok.txt проанализирован через natasha и результат сохранен в новом TXT файле
book1876_ok.txt проанализирован через natasha и результат сохранен в новом TXT файле
book1882_ok.txt проанализирован через natasha и результат сохранен в новом TXT файле
book1887_ok.txt проанализирован через natasha и результат сохранен в новом TXT файле
